In [344]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "06" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 23
repetitionName = "udrlBC20comma005"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.2
print(len(savedResults))
print(len(savedResults)/9)

25
2.7777777777777777


In [345]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 39)
(10, 39)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.716667,0.741667,0.725,0.716667,0.75,0.716667,0.716667,0.741667,0.725,0.716667,0.741667,0.725,0.716667,0.741667,0.725,0.716667,0.741667,0.725,0.741667,0.741667,0.725,0.716667,0.716667,0.691667,0.691667,0.725,0.708333,0.716667,0.741667,0.725,0.725,0.725,0.725,0.733333,0.725,0.716667,0.733333,0.733333,0.708333
2,0.6875,0.701389,0.694444,0.6875,0.694444,0.6875,0.6875,0.694444,0.6875,0.694444,0.6875,0.680556,0.680556,0.6875,0.680556,0.6875,0.701389,0.694444,0.6875,0.6875,0.694444,0.680556,0.701389,0.666667,0.680556,0.694444,0.659722,0.6875,0.694444,0.694444,0.6875,0.694444,0.694444,0.6875,0.680556,0.666667,0.694444,0.701389,0.680556
3,0.722222,0.712963,0.675926,0.722222,0.722222,0.675926,0.722222,0.703704,0.675926,0.722222,0.703704,0.675926,0.722222,0.712963,0.675926,0.722222,0.712963,0.675926,0.712963,0.703704,0.685185,0.731481,0.722222,0.675926,0.722222,0.722222,0.685185,0.722222,0.712963,0.675926,0.722222,0.712963,0.685185,0.731481,0.694444,0.675926,0.712963,0.703704,0.685185
4,0.715278,0.715278,0.708333,0.708333,0.715278,0.708333,0.715278,0.708333,0.708333,0.722222,0.722222,0.708333,0.708333,0.715278,0.701389,0.715278,0.715278,0.708333,0.680556,0.6875,0.659722,0.722222,0.708333,0.701389,0.715278,0.6875,0.659722,0.708333,0.715278,0.708333,0.694444,0.715278,0.715278,0.701389,0.715278,0.694444,0.694444,0.701389,0.645833
5,0.777778,0.701389,0.722222,0.784722,0.708333,0.722222,0.777778,0.701389,0.722222,0.777778,0.708333,0.722222,0.770833,0.701389,0.722222,0.777778,0.701389,0.722222,0.784722,0.722222,0.722222,0.756944,0.715278,0.743056,0.75,0.756944,0.736111,0.770833,0.701389,0.722222,0.777778,0.701389,0.722222,0.75,0.729167,0.715278,0.743056,0.75,0.736111
6,0.712121,0.689394,0.651515,0.712121,0.689394,0.651515,0.712121,0.689394,0.651515,0.712121,0.689394,0.651515,0.719697,0.689394,0.651515,0.712121,0.689394,0.651515,0.712121,0.689394,0.659091,0.69697,0.674242,0.659091,0.666667,0.643939,0.659091,0.719697,0.689394,0.651515,0.704545,0.689394,0.651515,0.704545,0.681818,0.659091,0.69697,0.681818,0.651515
7,0.645833,0.659722,0.652778,0.631944,0.652778,0.652778,0.652778,0.659722,0.652778,0.659722,0.666667,0.652778,0.645833,0.652778,0.652778,0.645833,0.659722,0.652778,0.666667,0.659722,0.638889,0.652778,0.659722,0.652778,0.666667,0.666667,0.625,0.645833,0.659722,0.652778,0.659722,0.666667,0.652778,0.645833,0.659722,0.645833,0.652778,0.659722,0.631944
8,0.75,0.7,0.7125,0.7875,0.7,0.7125,0.7625,0.7,0.7125,0.7625,0.7,0.7,0.775,0.7,0.7125,0.75,0.7,0.7125,0.7625,0.7,0.7125,0.7375,0.7,0.6875,0.75,0.7,0.6875,0.7625,0.7,0.7125,0.75,0.7,0.7125,0.75,0.7,0.7,0.75,0.7,0.7
9,0.677083,0.6875,0.6875,0.6875,0.6875,0.697917,0.6875,0.677083,0.708333,0.697917,0.697917,0.6875,0.6875,0.677083,0.6875,0.677083,0.6875,0.697917,0.697917,0.708333,0.697917,0.6875,0.6875,0.65625,0.6875,0.71875,0.645833,0.6875,0.6875,0.6875,0.6875,0.666667,0.677083,0.677083,0.6875,0.6875,0.697917,0.6875,0.697917


In [346]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.79 : subject 8, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--fftData, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:dataGCV-BC-fftDataBC-dataHRCV-BC--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--fftData, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:dataGCV-BC-fftDataBC-dataWCV-BC--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--welchData, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--dataHR, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:dataGCV-BC-fftDataBC--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:dataGCV-BC-dataGCVBC-fftDataBC--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.78 : subject 8, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--dataCorr1d, kernellinear, C:0.5
Accuracy 0.77 : subject 5, feature:dataGCV-BC-gaussianData-fftDataBC--dataFFT

In [347]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.79 : subject 8, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--fftData, kernellinear, C:0.5
Accuracy 0.78 : subject 8, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--dataCorr1d, kernellinear, C:0.5

Accuracy 0.78 : subject 5, feature:dataGCV-BC-fftDataBC-dataHRCV-BC--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--fftData, kernellinear, C:0.5

Accuracy 0.75 : subject 1, feature:dataGCV-BC-fftDataBC-dataFFTCV-BC--fftData, kernelsigmoid, C:2.5
Accuracy 0.74 : subject 1, feature:dataGCV-BC-fftDataBC--dataFFTCV-BC, kernelsigmoid, C:2.5

Accuracy 0.73 : subject 3, feature:dataGCV-BC-welchDataBC-fftDataBC--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.73 : subject 3, feature:dataGCV-BC-gaussianDataBC-fftDataBC--dataFFTCV-BC, kernellinear, C:0.5

Accuracy 0.72 : subject 4, feature:dataGCV-BC-welchDataBC-fftDataBC--dataFFTCV-BC, kernellinear, C:0.5
Accuracy 0.72 : subject 4, feature:dataGCV-BC-fftDataBC-dataF

In [348]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
# print(headers[maxInd])
# print(nrOfSubjects[maxInd])
# print(list(nameDict.keys())[combinationList[maxInd][0]])
# print(list(kernelDict.keys())[combinationList[maxInd][1]])
# print(f"C: {combinationList[maxInd][2]}")


[0.         0.72478632 0.68803419 0.70394112 0.70388177 0.7366453
 0.68123543 0.6531339  0.72115385 0.6875    ]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,0.711609,0.701033,0.692247,0.71539,0.702217,0.691706,0.714927,0.697304,0.69379,0.718399,0.701934,0.689314,0.714071,0.697561,0.689932,0.711609,0.701033,0.693404,0.71629,0.700005,0.68833,0.70918,0.698373,0.681591,0.703395,0.701719,0.674055,0.713454,0.700262,0.692247,0.712079,0.696867,0.69289,0.709018,0.697054,0.684601,0.708434,0.702095,0.681933


Max average accuracy 0.7183992704826039


In [349]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboAverage.argsort()[0][-5:][::-1]]
top5Vals = comboAverage[0][comboAverage.argsort()[0][-5:][::-1]]
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

dataGCV-BC-fftDataBC-dataFFTCV-BC--dataHR
linear
0.7183992704826039

dataGCV-BC-fftDataBC-dataHRCV-BC--dataFFTCV-BC
linear
0.7162902169846616

dataGCV-BC-fftDataBC-dataFFTCV-BC--fftData
linear
0.7153900112233447

dataGCV-BC-fftDataBC-dataFFTCV-BC--welchData
linear
0.7149270482603816

dataGCV-BC-fftDataBC-dataFFTCV-BC--dataCorr1d
linear
0.714071268237935



In [350]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

dataGCV-BC-fftDataBC--dataFFTCV-BC
1
dataGCV-BC-fftDataBC-dataFFTCV-BC--fftData
2
dataGCV-BC-fftDataBC-dataFFTCV-BC--welchData
3
dataGCV-BC-fftDataBC-dataFFTCV-BC--dataHR
4
dataGCV-BC-fftDataBC-dataFFTCV-BC--dataCorr1d
5
dataGCV-BC-fftDataBC-dataWCV-BC--dataFFTCV-BC
6
dataGCV-BC-fftDataBC-dataHRCV-BC--dataFFTCV-BC
7
dataGCV-BC-welchDataBC-fftDataBC--dataFFTCV-BC
8
dataGCV-BC-dataHRBC-fftDataBC--dataFFTCV-BC
9
dataGCV-BC-gaussianData-fftDataBC--dataFFTCV-BC
10
dataGCV-BC-dataGCVBC-fftDataBC--dataFFTCV-BC
11
dataGCV-BC-gaussianDataBC-fftDataBC--dataFFTCV-BC
12
dataCorr1dBC-dataGCV-BC-fftDataBC--dataFFTCV-BC
13


In [351]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38
0,6.404482,6.309301,6.230219,6.43851,6.319949,6.225358,6.434343,6.275737,6.244108,6.465593,6.317403,6.20383,6.426641,6.278051,6.209386,6.404482,6.309301,6.240636,6.446612,6.300042,6.194971,6.382618,6.285354,6.134322,6.330556,6.315467,6.066498,6.421086,6.302357,6.230219,6.408712,6.271801,6.236006,6.381166,6.273485,6.161406,6.375905,6.318855,6.137395


9
9.0
Max total significant accuracy then averaged 0.7183992704826039 
dataGCV-BC-fftDataBC-dataFFTCV-BC--dataHR
linear
C: 0.5 


In [352]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
